# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,304.54,82,40,6.69,MP,1731637944
1,1,adamstown,-25.0660,-130.1015,296.46,87,80,3.93,PN,1731637689
2,2,grytviken,-54.2811,-36.5092,273.87,68,86,4.52,GS,1731637946
3,3,port mathurin,-19.6833,63.4167,297.45,75,10,7.46,MU,1731637947
4,4,colares,38.7992,-9.4469,287.53,83,54,8.18,PT,1731637948


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
coastline = gv.feature.coastline.opts(width=1000, height=500, bgcolor="lightblue") * gv.feature.land.opts(color="whitesmoke")
points = gv.Points(city_data_df, kdims=["Lon", "Lat"], vdims=["City", "Humidity"]).opts(color="City", cmap="Category10", legend_position="right", tools=["hover"], size=(5 - (gv.dim("Humidity") * .04)))

coastline * points

:Overlay
   .Coastline.I :Feature   [Longitude,Latitude]
   .Land.I      :Feature   [Longitude,Latitude]
   .Points.I    :Points   [Lon,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
unclean_narrowed_city_data_df = city_data_df[(city_data_df["Humidity"] <= 25) & (city_data_df["Cloudiness"] <= 50) & (city_data_df["Wind Speed"] <= 10)]
# Drop any rows with null values
narrowed_city_data_df = unclean_narrowed_city_data_df.dropna()
# Display sample data
narrowed_city_data_df

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,cootamundra,-34.6500,148.0333,302.58,17,3,3.78,AU,1731638025
77,77,alice springs,-23.7000,133.8833,303.90,13,33,3.60,AU,1731638038
98,98,las vegas,36.1750,-115.1372,288.53,24,0,4.12,US,1731637863
144,144,faya-largeau,17.9257,19.1043,293.16,21,0,7.67,TD,1731638119
154,154,bardai,11.6500,7.2333,292.58,23,0,4.51,NG,1731638131
309,309,zinder,13.8072,8.9881,295.71,19,0,4.31,NE,1731638466
384,384,nagqu,31.5000,92.0000,275.80,14,2,3.44,CN,1731638560
397,397,ouallam,14.3188,2.0866,298.82,25,0,4.52,NE,1731638576
406,406,tchintabaraden,15.8969,5.7985,296.65,21,25,4.09,NE,1731638588
494,494,deming,32.2687,-107.7586,285.09,22,0,2.06,US,1731638691


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_data_df[["City", "Country", "Lat", "Lon", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df

,City,Country,Lat,Lon,Humidity,Hotel Name
67,cootamundra,AU,-34.6500,148.0333,17,None
77,alice springs,AU,-23.7000,133.8833,13,None
98,las vegas,US,36.1750,-115.1372,24,None
144,faya-largeau,TD,17.9257,19.1043,21,None
154,bardai,NG,11.6500,7.2333,23,None
309,zinder,NE,13.8072,8.9881,19,None
384,nagqu,CN,31.5000,92.0000,14,None
397,ouallam,NE,14.3188,2.0866,25,None
406,tchintabaraden,NE,15.8969,5.7985,21,None
494,deming,US,32.2687,-107.7586,22,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": None,
    "bias": None
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat = row["Lat"]
    city_lon = row["Lon"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lon},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lon},{city_lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    url = f"{base_url}?categories=accommodation.hotel&filter={params["filter"]}&bias={params["bias"]}&apiKey={geoapify_key}"
    response = requests.get(url)

    # Convert the API response to JSON format
    hotel_data = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cootamundra - nearest hotel: Globe Hotel
alice springs - nearest hotel: Aurora Alice Springs
las vegas - nearest hotel: Downtown Grand
faya-largeau - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
zinder - nearest hotel: Hotel Zentral
nagqu - nearest hotel: Naqu Hotel
ouallam - nearest hotel: No hotel found
tchintabaraden - nearest hotel: No hotel found
deming - nearest hotel: No hotel found
calama - nearest hotel: Apart Hotel Geotel Calama
deqen - nearest hotel: No hotel found


,City,Country,Lat,Lon,Humidity,Hotel Name
67,cootamundra,AU,-34.6500,148.0333,17,Globe Hotel
77,alice springs,AU,-23.7000,133.8833,13,Aurora Alice Springs
98,las vegas,US,36.1750,-115.1372,24,Downtown Grand
144,faya-largeau,TD,17.9257,19.1043,21,No hotel found
154,bardai,NG,11.6500,7.2333,23,No hotel found
309,zinder,NE,13.8072,8.9881,19,Hotel Zentral
384,nagqu,CN,31.5000,92.0000,14,Naqu Hotel
397,ouallam,NE,14.3188,2.0866,25,No hotel found
406,tchintabaraden,NE,15.8969,5.7985,21,No hotel found
494,deming,US,32.2687,-107.7586,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
coastline = gv.feature.coastline.opts(width=1000, height=300, bgcolor="lightblue") * gv.feature.land.opts(color="whitesmoke")
points = gv.Points(hotel_df, kdims=["Lon", "Lat"], vdims=["City", "Humidity", "Hotel Name", "Country"]).opts(color="City", cmap="Category10", legend_position="right", tools=["hover"], size=(5 - (gv.dim("Humidity") * .04)))

coastline * points

:Overlay
   .Coastline.I :Feature   [Longitude,Latitude]
   .Land.I      :Feature   [Longitude,Latitude]
   .Points.I    :Points   [Lon,Lat]   (City,Humidity,Hotel Name,Country)